In [ ]:
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.backends import cudnn
import os
import logging
from torch.utils.data import DataLoader
if not os.path.isdir('a'):
    !git clone https://github.com/pI88Id/MLProject-Incremental.Learning.git
    os.rename("MLProject-Incremental.Learning","a")

import torchvision
from torchvision import transforms
from torchvision.models import resnet18

from a.Cifar100.Cifar100 import Cifar100
from Cifar100 import utils

**Set Arguments**

In [ ]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 100
CLASSES_EACH_TRAIN = 10

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                    # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 5  #30    # Total number of training epochs (iterations over dataset)
STEP_SIZE = 2  #30    # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 100

**Define Data Preprocessing**

In [ ]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
                                    ])

# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])

**Prepare Network**

In [ ]:
net = resnet18()
best_net = resnet18()

net.fc = nn.Linear(net.fc.in_features, NUM_CLASSES)

**Prepare Training**

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

parameters_to_optimize = net.parameters()

optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
# optimizer = optim.Adam(parameters_to_optimize, lr=LR, weight_decay=WEIGHT_DECAY)

# Define scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train and Test**

In [ ]:
net = net.to(DEVICE)
cudnn.benchmark = True # Calling this optimizes runtime

current_step = 0
best_accuracy = 0
loss = 0
index = 0
accuracy_train = []
accuracy_test = []
loss_train = []
loss_test = []

In [ ]:
#New variable from our class Cifar100
# cifar100 = Cifar100(BATCH_SIZE, NUM_EPOCHS, DEVICE, LR, STEP_SIZE, GAMMA)

for index in range(0, int(NUM_CLASSES/CLASSES_EACH_TRAIN)):

  print('index', index)
  #Load data from Cifar100
  train_dataset = utils.Cifar100(classes=range(10), train=True, transform=train_transform)
  test_dataset = utils.Cifar100(classes=range(10), train=False, transform=train_transform)
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8, drop_last=True)
  test_dataloader = DataLoader(test_dataset, batch_size=int(BATCH_SIZE), shuffle=False, num_workers=8) #Batch_size decrease for GPU ram problems

  # train_dataloader = cifar100.load('train', index=index)
  # test_dataloader = cifar100.load('test', index=index)


  # Start iterating over the epochs
  for epoch in range(NUM_EPOCHS):